In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_excel('Load Data in kW.xlsx')
print(data.shape)
data.head()

(2184, 3)


,Date,Time,Load (kW)
0,01.09.2018,00:00:00,5551.82208
1,NaN,01:00:00,4983.17184
2,NaN,02:00:00,4888.39680
3,NaN,03:00:00,5072.95872
4,NaN,04:00:00,5196.25980


In [3]:
x = []
y = []
for i in range(1, data.shape[0]):
    x.append(data['Load (kW)'].iloc[i-1])
    y.append(data['Load (kW)'].iloc[i])
    
data = pd.DataFrame({'Previous Hour Load' : x, 'Present Hour Load' : y})

maxX = np.max(data['Previous Hour Load'])
minX = np.min(data['Previous Hour Load'])
maxY = np.max(data['Present Hour Load'])
minY = np.min(data['Present Hour Load'])

data['Previous Hour Load'] = (data['Previous Hour Load'] - minX) / (maxX - minX)
data['Present Hour Load'] = (data['Present Hour Load'] - minY) / (maxY - minY)

In [4]:
x = data['Previous Hour Load']
y = data['Present Hour Load']
trainX, testX, trainY, testY = train_test_split(x, y, test_size=0.1, random_state=101)

In [5]:
m = np.random.uniform(-5, 5)
c = np.random.uniform(-5, 5)
eta = 0.01
epochs = 500
vm = 0
vc = 0
v = 0.1

# Loop for epochs
for _ in range(epochs):
    
    for i in range(trainX.shape[0]):

        tempM = m + v*vm
        tempC = c + v*vc
        
        gradM = -1 * (trainY.iloc[i] - tempM * trainX.iloc[i] - tempC) * trainX.iloc[i]
    
        gradC = -1 * (trainY.iloc[i] - tempM * trainX.iloc[i] - tempC)
        
        vm = (v * vm) - (eta * gradM)
        vc = (v * vc) - (eta * gradC)
        
        m += vm
        c += vc

In [6]:
train_pred = [m * trainX.iloc[i] + c for i in range(trainX.shape[0])]
test_pred = [m * testX.iloc[i] + c for i in range(testX.shape[0])]

train_pred = [i * (maxY - minY) + minY for i in train_pred]
test_pred = [i * (maxY - minY) + minY for i in test_pred]
trainY = [i * (maxY - minY) + minY for i in trainY]
testY = [i * (maxY - minY) + minY for i in testY]

training_MAE = sum([abs(train_pred[i] - trainY[i]) for i in range(len(trainY))]) / len(trainY)
training_MSE = sum([(train_pred[i] - trainY[i]) ** 2 for i in range(len(trainY))]) / len(trainY)
training_RMSE = training_MSE ** 0.5
print('Training Error :')
print(f'MAE : {training_MAE}')
print(f'MSE : {training_MSE}')
print(f'RMSE : {training_RMSE}')

testing_MAE = sum([abs(test_pred[i] - testY[i]) for i in range(len(testY))]) / len(testY)
testing_MSE = sum([(test_pred[i] - testY[i]) ** 2 for i in range(len(testY))]) / len(testY)
testing_RMSE = testing_MSE ** 0.5
print('\nTesting Error :')
print(f'MAE : {testing_MAE}')
print(f'MSE : {testing_MSE}')
print(f'RMSE : {testing_RMSE}')

Training Error :
MAE : 470.78544515956565
MSE : 430838.4062024668
RMSE : 656.3828198562686

Testing Error :
MAE : 516.1635560852443
MSE : 542768.9860455303
RMSE : 736.7285701298208


In [7]:
x = float(input('Enter the load at previous hour : '))
x = (x - minX) / (maxX - minX)
prediction = m * x + c
prediction = (prediction * (maxY - minY)) + minY
print('Predicted load at present hour :', prediction)

Enter the load at previous hour : 2453
Predicted load at present hour : 3268.792159626758
